# Introduction to Performance Measurement

1. Extract data
2. Work with the data

Now moving to backtesting. Backtesting is very widely used although it has limitations and crticisms. It uses the past to predict the future, which in itself is a flaw and not grounded. Secondly, backtesting does not take into account slippage, trading costs, drawdowns, etc. 
When we devise a strategy to run on the market, then we test it on historical data to see how it performs, and so we may be able to quantify its risks and rewards.

# Compounded Annual Growth rate

CAGR is arguably the most important metric used by one and all in quant. It speaks of per year return of the value of an asset. Works similar to Compound Interest, where we want to estimate the rate. 
Uses cumulative return.

In [1]:
# Get the Data 

import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt 

In [11]:
tickers = ["AMZN","MSFT","GOOG"]
ohlcv_data = {}

for ticker in tickers:
    ohlcv_data[ticker] = yf.download(ticker, period = "6mo", interval = "1d")
    

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


### Simple Buy and Hold Strategy !

In [24]:
def CAGR(DF):
    df = DF.copy()
    df["return"] = df["Adj Close"]/df["Adj Close"].shift(1) - 1
    df["cumret"] = (1+df["return"]).cumprod()
    n = len(df)/252 # no of trading days in a year.. rule of thumb
    CAGR = (df["cumret"].iloc[-1])**(1/n) - 1
    return CAGR

In [25]:
for ticker in tickers:
    print(f"CAGR for {ticker} is :", CAGR(ohlcv_data[ticker]))

CAGR for AMZN is : 0.611326423251572
CAGR for MSFT is : 0.341533443186282
CAGR for GOOG is : 0.7078063823314968


# Volatility  

Standard deviation of results. Changes with different timescales : hours, seconds, days, months all have different volatility. To compare apples with apples, we multiply daily data with $\sqrt{252}$. Single most used quantifier for risk.
Assumes a normal distribution to returns. Not the case many times(think turbulence). So, tail risk is not captured - extreme events.

In [40]:
def volatility(DF):
    df = DF.copy()
    volatility = df["Adj Close"].pct_change().std() * np.sqrt(252)
    return volatility

In [41]:
volatility(ohlcv_data["AMZN"])

0.24096053058531944

# Sharpe Ratio

talks about both returns and risk. Most used JPI metric in the industry. 
Def : The average return earned in excess of the risk free rate per unit of volatility. If you dont want to take any risk, then you end up with the sharpe ratio.

\begin{equation}
Sharpe Ratio = \frac{R_p - R_f}{\sigma_p}
\end{equation}

how to calculate Risk free rate - govt. bonds in US, guilts in UK, usually constant. 


In [42]:
def sharpe_ratio(stock, rf = 0.03):
    rp = CAGR(stock)
    σ_p = volatility(stock) 
    return (rp - rf)/σ_p  # x - μ/σ style 

In [43]:
# for sortino, we need to remove positive volatility 
# we use regular cagr for sortino. onpy σ changes

def sortino(df, rf = 0.03):
    returns = df["Adj Close"].pct_change()
    returns_neg = np.where(returns < 0 ,returns, 0)
    returns_neg = pd.Series(returns_neg[returns_neg!=0]) # no need to convert to nan values 
    n = len(returns)/252
    rp = CAGR(df) 
    σ_p = returns_neg.std() * np.sqrt(252)
    return (rp - rf)/σ_p

In [45]:
for ticker in tickers:
    print(f"Sharpe {ticker} : {sharpe_ratio(ohlcv_data[ticker])}")
    print(f"Sortino {ticker} : {sortino(ohlcv_data[ticker])}")

Sharpe AMZN : 2.4125379448636948
Sortino AMZN : 4.86968140165257
Sharpe MSFT : 1.6461399063001732
Sortino MSFT : 2.231099816897745
Sharpe GOOG : 2.4658527138036916
Sortino GOOG : 3.2824802071572554


# Maximum Drawdown, Calmar ratio 

Largest % drop in asset price over a specified time period. Peak - trough. Peak must come before trough. Important risk metric. 

In [53]:
# how to find max drawdown? 
def max_drawdown(df):
    returns = df["Adj Close"].pct_change()
    cum_returns = (returns + 1).cumprod()
    #cum_returns.plot()
    cum_roll_max = cum_returns.cummax()
    drawdown = cum_roll_max - cum_returns # wow why does this work?
    drawdown /= cum_roll_max
    return drawdown.max()

In [54]:
def calmar(DF):
    df = DF.copy()
    return CAGR(df)/max_drawdown(df)

In [55]:
for ticker in tickers:
    print(f"{ticker} max drawdown is {max_drawdown(ohlcv_data[ticker])} ")
    print(f"{ticker} calmar ratio is {calmar(ohlcv_data[ticker])} ")

AMZN max drawdown is 0.08135416363152236 
AMZN calmar ratio is 7.514383972041733 
MSFT max drawdown is 0.09325294654102148 
MSFT calmar ratio is 3.662441304587016 
GOOG max drawdown is 0.14389052550991255 
GOOG calmar ratio is 4.919061764652019 


# DONEDONEDONE 